In [ ]:
import requests # 온라인상의 데이터 수집. webbrowser (URL 요청) ---> Backend(웹서버) (응답)---(JSON)--->
from bs4 import BeautifulSoup
from pandas import DataFrame

In [ ]:
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&listSize=72&channel=user"
userAgent ="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
session = requests.Session()
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent,
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"
})
c = session.get(url)    

if c.status_code != 200:
    msg = "[%d Error] %s 에러가 발생함" % (c.status_code, c.reason)
    raise Exception(msg)

c.encoding = 'utf-8'
soup = BeautifulSoup(c.text)
soup

In [ ]:
searchList = soup.select('.search-product')
searchList


In [219]:
result = []

for product in searchList:
    nameEl = product.select('.name')
    name = nameEl[0].text.strip()
    basePriceEl = product.select('.base-price')
    if not basePriceEl:
        basePrice = None
    else: 
        basePrice = basePriceEl[0].text.strip()
        basePrice = basePrice.replace(",","")
    salePriceEl = product.select('.price-value')
    if not salePriceEl:
        salePrice = None
    else: 
        salePrice = salePriceEl[0].text.strip()
        salePrice = salePrice.replace(",","")
    discountEl = product.select('.instant-discount-rate')
    if not discountEl:
        discount = None
    else:
        discount = discountEl[0].text.strip()
    rateEl = product.select('.star')
    if not rateEl:
        rate = None
    else:
        rate = rateEl[0].text.strip()
    reviewcountEl = product.select('.rating-total-count')
    if not reviewcountEl:
        reviewcount = None
    else:
        reviewcount = reviewcountEl[0].text.strip()
        reviewcount = int(reviewcount[1:-1])
    # 중고상품 판별 조건
    if not basePrice and not salePrice:
        continue

    item = { "이름": name,
                "원가": basePrice, 
                "할인율": discount, 
                "할인가": salePrice,
                "별점": rate, 
                "리뷰수":reviewcount}
    result.append(item)

   
        


In [220]:
df = DataFrame(result)
df.to_excel('쿠팡에서 노트북 정보.xlsx')
df

,이름,원가,할인율,할인가,별점,리뷰수
0,LG 그램 15인치 16인치 17인치 12세대 i5 i7 512GB 16G 일반 2...,None,None,1499000,5.0,34.0
1,"베이직스 2022 베이직북 14 3세대, BB1422SS, 256GB, 화이트, W...",None,None,398000,4.5,1127.0
2,"레노버 2022 아이디어패드 노트북, 아틱그레이, SLIM3 15ITL6, 코어i7...",None,None,629000,5.0,351.0
3,"LG전자 2022 울트라 PC 엣지 16, 256GB, 차콜 그레이, 16U70Q-...",1800000,25%,1345500,5.0,60.0
4,"삼성노트북9 Metal 15인치 코어i5 SSD 512GB 윈도우10, 단품, 단품",1396000,69%,419000,5.0,5.0
...,...,...,...,...,...,...
67,"삼성 사무용 인강 노트북 인텔 i5 i7 SSD장착 윈10 홈, NT370E, WI...",None,None,339000,4.5,13.0
68,"레노버 씽크패드 T470s, WIN10 Pro, 16GB, 256GB, 코어i7, 블랙",1000000,67%,325000,None,NaN
69,삼성 사무용 게이밍 노트북 롤 포토샵 윈10 pro 탑재 SSD i5 (NT371B...,None,None,369000,5.0,3.0
70,"레노버 2020 IdeaPad S150 11.6, 아이스 블루, 셀러론, 64GB,...",439860,32%,299000,4.5,637.0
